<a href="https://colab.research.google.com/github/prad69/NLP/blob/main/Roberta_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Introduction to RoBERTa**

1. RoBERTa is a reimplementation of BERT with some changes in the key
   hyperparameters and minor embedding tweaks. It uses a byte-level BPE as a tokenizer and a different pretraining scheme.
2. RoBERTa is trained for longer sequences i.e. the number of iterations
   is increased from 100K to 300K and then further to 500K.
3. RoBERTa uses larger byte-level BPE vocabulary with 50K subword units instead
   of character-level BPE vocabulary of size 30K used in BERT.
4. In the Masked Language Model (MLM) training objective, RoBERTa employs dynamic masking to generate the masking pattern every time a sequence is fed to the model.
5. RoBERTa doesn’t use token_type_ids, and we don’t need to define which token
   belongs to which segment. Only separate segments with the separation token tokenizer.sep_token (or ).
6. Larger mini-batches and learning rates are used in RoBERTa’s training.
7. NSP is removed from its objective.

## Uncomment below lines , only require to mount notebook to a given directory and persist the model in **google** drive

In [1]:
# mounting the drive
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
# # Path to the directory where this model will be saved. If you want to save it to another path then link it with google drive.
!cd  drive/MyDrive/Colab Notebooks

/bin/bash: line 1: cd: too many arguments


In [3]:
# Installing the required libraries
!pip install transformers[torch]

In [4]:
# Accelerate is a library that enables the same PyTorch code to be run across any distributed configuration.
!pip install accelerate -U

In [5]:
# Cloning the dataset tweeteval repo here
!git clone https://github.com/cardiffnlp/tweeteval /tmp/tweeteval

Cloning into '/tmp/tweeteval'...
remote: Enumerating objects: 370, done.
remote: Counting objects: 100% (16/16), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 370 (delta 13), reused 3 (delta 1), pack-reused 354 (from 1)
Receiving objects: 100% (370/370), 8.49 MiB | 7.50 MiB/s, done.
Resolving deltas: 100% (122/122), done.


In [6]:
#1.
# defining the roberta base model here
from transformers import pipeline

roberta_base_model = pipeline(
                              "fill-mask",
                              model = "roberta-base",
                              tokenizer = "roberta-base"
                             )


config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cuda:0


In [7]:
#2.
# function to predict masked token in sentence
def predict_mask_token(model, sentence):
  predictions = model(sentence)
  for prediction in predictions:
    print(prediction['sequence'].strip('<s>').strip('</s>'), end='\t--- ')
    print(f"{round(100*prediction['score'],2)}% confidence")

In [8]:
#3.
# function call with parameters roberta base model and sentence with mask token
predict_mask_token(roberta_base_model, "Send these <mask> back!")

Send these pictures back!	--- 16.66% confidence
Send these photos back!	--- 10.79% confidence
Send these emails back!	--- 7.67% confidence
Send these images back!	--- 4.86% confidence
Send these letters back!	--- 4.84% confidence


In [9]:
#4.
predict_mask_token(roberta_base_model, "Elon Musk is the founder of <mask>")

Elon Musk is the founder of Tesla	--- 69.61% confidence
Elon Musk is the founder of SpaceX	--- 29.53% confidence
Elon Musk is the founder of PayPal	--- 0.73% confidence
Elon Musk is the founder of Twitter	--- 0.05% confidence
Elon Musk is the founder of Facebook	--- 0.02% confidence


**Configuring, tokenizing, training & saving the model**

Preparation of dataset(i.e train data converted to tokens with a block size of 512) for training model.

In [10]:
#5.
from transformers import RobertaTokenizer, RobertaForMaskedLM
# constructs a RoBERTa BPE tokenizer
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
# configuring the model and loading the model weights
model = RobertaForMaskedLM.from_pretrained('roberta-base')

In [11]:
#6.
from transformers import LineByLineTextDataset
# Preparing the dataset for training model
dataset = LineByLineTextDataset(
    tokenizer=tokenizer, # for converting train data into tokens
    file_path="/tmp/tweeteval/datasets/hate/train_text.txt", # path where train data file exists
    block_size=512,
)

/usr/local/lib/python3.12/dist-packages/transformers/data/datasets/language_modeling.py:119: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


Here first we will understand-

**What is Data Collator and what it does?**

Data collators are objects that will form a batch by using a list of dataset elements as input. These elements are of the same type as the elements of train_dataset or eval_dataset.
To be able to build batches, data collators may apply some processing (like padding). **DataCollatorForLanguageModeling** also apply some random data augmentation (like random masking) on the formed batch.

Here we will make use of **DataCollatorForLanguageModeling** function for masking randomly 15% of the tokens for MLM task. Parameters are

**tokenizer-** The tokenizer used for encoding the data,

**mlm-** Whether or not to use masked language modeling. If set to False, the labels are the same as the inputs with the padding tokens ignored (by setting them to -100). Otherwise, the labels are -100 for non-masked tokens and the value to predict for the masked token.

**mlm_probability-** The probability with which to (randomly) mask tokens in the input, when mlm is set to True.

In [12]:
#7.
# importing the library DataCollatorForLanguageModeling
from transformers import DataCollatorForLanguageModeling
# For Mask Language Modelling(MLM) task
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15 # for masking randomly 15% of the tokens for MLM task
)

**Note-**

 **Difference between RoBERTa base model and RoBERTa retrained model?**

 RoBERTa base model is very generic model and when we are training this model on a new database then after getting trained it is RoBERTa retrained model.

In [13]:
#8.
# training the model
from transformers import Trainer, TrainingArguments

# initalizing the training arguments
training_args = TrainingArguments(
    output_dir="./roberta-retrained",  # path where roberta retrained model will be saved.
    overwrite_output_dir=True, # permission for overwriting the output directory
    num_train_epochs=1, # num_train_epochs is a hyperparameter that defines the number of times the learning algorithm will work through the entire training dataset.
    per_device_train_batch_size=48, # The batch size per GPU/TPU core/CPU for training.
    seed=1 # Random seed for initialization. This is optional and defaults to 42.
)

# Passing the model, training arguments, data collator(for masking tokens), dataset to trainer function for training the model
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset
)

In [14]:
#9.
# model is getting trained here
trainer.train()
# saving the model
trainer.save_model("roberta-retrained/model")

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: pradeep6923 (pradeep6923-pp) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss


In [15]:
#10.
# loading the trained model
roberta_retrained_model = pipeline(
                                    "fill-mask",
                                    model="roberta-retrained/model",
                                    tokenizer="roberta-base"
                                )

Device set to use cuda:0


In [16]:
#11.
# Predicting the masked token in sentence with trained model by function call
predict_mask_token(roberta_retrained_model, "Send these <mask> back!")

Send these guys back!	--- 13.11% confidence
Send these pictures back!	--- 5.38% confidence
Send these men back!	--- 3.57% confidence
Send these pics back!	--- 2.82% confidence
Send these photos back!	--- 2.42% confidence


In [17]:
predict_mask_token(roberta_retrained_model, "I hate watching <mask> sports")

I hate watching fantasy sport	--- 11.8% confidence
I hate watching college sport	--- 6.49% confidence
I hate watching violent sport	--- 4.89% confidence
I hate watching live sport	--- 4.23% confidence
I hate watching stupid sport	--- 4.08% confidence


In [18]:
predict_mask_token(roberta_base_model, "I hate watching <mask> sports")

I hate watching fantasy sport	--- 24.12% confidence
I hate watching college sport	--- 10.72% confidence
I hate watching live sport	--- 9.0% confidence
I hate watching professional sport	--- 7.1% confidence
I hate watching pro sport	--- 3.46% confidence


In [19]:
predict_mask_token(roberta_retrained_model, "Hello I'm a <mask> model.")

Hello I'm a fashion model.	--- 15.65% confidence
Hello I'm a male model.	--- 12.82% confidence
Hello I'm a female model.	--- 3.83% confidence
Hello I'm a Russian model.	--- 2.62% confidence
Hello I'm a beauty model.	--- 2.46% confidence


In [20]:
predict_mask_token(roberta_base_model, "Hello I'm a <mask> model.")

Hello I'm a male model.	--- 33.07% confidence
Hello I'm a female model.	--- 4.66% confidence
Hello I'm a professional model.	--- 4.23% confidence
Hello I'm a fashion model.	--- 3.72% confidence
Hello I'm a Russian model.	--- 3.25% confidence


In [21]:
predict_mask_token(roberta_retrained_model, "The man worked as a <mask>.")

The man worked as a waiter.	--- 8.26% confidence
The man worked as a mechanic.	--- 6.16% confidence
The man worked as a cop.	--- 5.16% confidence
The man worked as a prostitute.	--- 3.51% confidence
The man worked as a butcher.	--- 3.46% confidence


In [22]:
predict_mask_token(roberta_base_model, "The man worked as a <mask>.")

The man worked as a mechanic.	--- 8.7% confidence
The man worked as a waiter.	--- 8.2% confidence
The man worked as a butcher.	--- 7.33% confidence
The man worked as a miner.	--- 4.63% confidence
The man worked as a guard.	--- 4.02% confidence
